# Hyperdimensional Computing

In [ ]:
# Setup: Install kongming-hv (required for Google Colab)
# After running this cell, restart the runtime: Runtime → Restart runtime
!pip install -q kongming-hv

This notebook serves as an illustration of my work on high-dimensional computing (HDC).

Any questions / comments, pleast contact me (Kevin Yang, yangzh@gmail.com)

To get started, first we need to import related Python modules/packages into this Notebook.

## The `kongming.api` module

In [1]:
from kongming import api

`kongming.api` is an auto-generated module from cross-language [`protocol buffer`](https://protobuf.dev) definitions.

For now, you just need to know a few constants for hypervector configurations:

* `api.MODEL_64K_8BIT`=1: used for hyper-vectors, where $N=65536$, and sparsity=$1/256$ (8-bit depth);
* `api.MODEL_1M_10BIT`=2: used for hyper-vectors, where $N=1M$, and sparsity=$1/1024$ (10-bit depth);
* `api.MODEL_16M_12BIT`=3: used for hyper-vectors, where $N=16M$, and sparsity=$1/4096$ (12-bit depth).

Later on you can either use these constants, or just the numeric value.

## `hv` module from `kongming` package

In [2]:
from kongming.hv import (
    Domain,
    Pod,
    Seed128,
    SparseOperation,
    SparseSegmented,
    Sparkle,
    Set,
    Sequence,
    Octopus,
    Learner,
    HyperBinary,
    d0,
    bind,
    bundle,
    inverse,
    overlap,
    hamming,
    equal,
    is_identity,
    to_message,
    from_message,
    keys,
)

Module `kongming.hv` contains useful abstractions and operations for hypervectors.

To get started, `SparseOperation` class, as its name suggests, models the sparse operation in general. It also provide pseudo-random number generators used in various operations.

Note the underlying implementation is written in Go, and exported as a Python module using `gopy`, with underlying translation layer in work. Unlike Python, which follows snake naming convention, the documentation may contain references to function names in Go (with Camel convention).

In [3]:
help(SparseOperation)

Help on class SparseOperation in module kongming.ext.hv:

class SparseOperation(kongming.ext.go.GoClass)
 |  SparseOperation(*args, **kwargs)
 |
 |  SparseOperation models operations for stochastic sparse hypervectors.
 |
 |  Method resolution order:
 |      SparseOperation
 |      kongming.ext.go.GoClass
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __del__(self)
 |
 |  __init__(self, *args, **kwargs)
 |      handle=A Go-side object is always initialized with an explicit handle=arg
 |      otherwise parameters can be unnamed in order of field names or named fields
 |      in which case a new Go object is constructed first
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  cardinality(self)
 |      Cardinality() int
 |
 |      Cardinality returns the associated cardinality (ON bit counts).
 |
 |  model(self)
 |      Model() int
 |
 |      Model returns the associated sparsity model.
 |
 |  rng(self)
 |      RNG() object


An `SparseOperation` instance can be created by `SparseOperation.create`: the second and third arguments are an initial seeds (2 uint64 seeds) for the internal random number generator. For now, any number will do.

In [4]:
so=SparseOperation.create(api.MODEL_1M_10BIT, 0, 99)

The associated model can be retrieved from this object, which is the numeric value for `MODEL_1M_10BIT`

In [5]:
so.model(), so.width()

(2, 1048576)

### Generating hypervectors

With the random number generator associated with `so`, a random hyper-vector can be generated by:

In [6]:
a=Sparkle.random(d0(), so)

The returned hyper-vector is of type `Sparkle`. 

`d0()` returns a default domain (as the conceptual collection of hypervectors), we will stay with default domain for now, and later on we can try custom domains.

Again, familiarize yourself with this class...

In [7]:
help(Sparkle)

Help on class Sparkle in module kongming.ext.hv:

class Sparkle(SparseSegmented)
 |  Sparkle(*args, **kwargs)
 |
 |  # Python type for struct hv.Sparkle
 |
 |  Method resolution order:
 |      Sparkle
 |      SparseSegmented
 |      kongming.ext.go.GoClass
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __del__(self)
 |
 |  __init__(self, *args, **kwargs)
 |      handle=A Go-side object is always initialized with an explicit handle=arg
 |      otherwise parameters can be unnamed in order of field names or named fields
 |      in which case a new Go object is constructed first
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  clone(self)
 |      Clone() object
 |
 |  exponent(self)
 |      Exponent() int
 |
 |  power(self, p)
 |      Power(int p) object
 |
 |  proto_load(self, ctx)
 |      ProtoLoad(object ctx) object, str
 |
 |  repr(self, opts)
 |      Repr(long opts) str
 |
 |  string(self)
 |      String() str
 |
 |  -

`Sparkle` inherits from `SparseSegmented`, which serves as the container for common hypervector attributes. One of such critical attributes is `stable_hash`: the signature hash value for this hypervector. Different vector, no matter how small the difference is, will produce dramatically different hash value. 

In addition, the design for this hash is representation-agnostic: the idea is that the hash value for the same vector, in different forms of representation, let it be `Sparkle` or any other form, will remain unchanged: it's always safe to compare their hash value to determine equality.

In addition, the `stable_hash` will be persisted during serialization / de-serialization.

In [8]:
hex(a.stable_hash())

'0xfac0717a98ae4125'

Another useful way to examine a hyper-vector is `string()`, essentially turn a vector into its human-readable string form.


In [9]:
a.string()

'✨:🫛0x..0d01'

The emoji of ✨ provides a visual hint that it is a `Sparkle` type. 🫛 indicates a randomly-generated seed (for this hyper-vector) and the last 4 hexadecimal digits is 0d01, as `Sparkle` is uniquely determined by its seeds. Note this is NOT the `stable_hash` value.

In [10]:
b=Sparkle.random(d0(), so)

Note the `SparseOperation` object will change its internal RNG status for each usage, and the next call of `Sparkle.random` (with the same `so` instance) will produce a complete new hyper-vector.

VSA predicts another random hypervector `b` should be almost orthogonal to `a`, as validated by their overlap.

In [11]:
overlap(a,b)

1

In the meanwhile, their Hamming distance will be big: they are really far distant apart.

In [12]:
hamming(a,b)

2046

Overlap and Hamming between 2 hypervectors here also follows: 

`2 * Overlap(a, b) + Hamming(a, b) = 2 * M`

`M=1024` here.

Another way to produce a hyper-vector is by directly specifying model, domain, and seed.

In [13]:
c=Sparkle.from_seed(api.MODEL_1M_10BIT, d0(), 1234)

The third argument is the uint64 seed for this hyper-vector: different seeds will produce uniquely differnt vectors.

In [14]:
c.string(), hex(1234)

('✨:🫛0x..04d2', '0x4d2')

As mentioned earlier, 🫛 indicates the seed value used to bootstrap this `Sparkle` instance.

It's also trivial to verify this random vector is almost orthogonal to previous `a` and `b`:

In [15]:
overlap(a,c), overlap(b, c)

(1, 0)

`hv` package also provides a few other shortcuts to produce hyper-vector deterministically.

In [16]:
d=Sparkle.from_word(api.MODEL_1M_10BIT, d0(), "random")
d.string()

'✨:🌱random'

Note 🌱 indicates the seed word used to bootstrap this `Sparkle` instance. Internally we bootstrap the generation with computed hash from the seed word.

In [17]:
overlap(a,d), overlap(b, d)

(0, 1)

In [18]:
e=Sparkle.from_word(api.MODEL_1M_10BIT, d0(), "RANDOM")
e.string()

'✨:🌱RANDOM'

Note seed words are case-sentitive so that `d` and `e` are quite different.

In [19]:
equal(d,e)

False

### Inspection and serializations for hypervectors

Remember `a` is of type `Sparkle`, and there might be other hypervector classes: they can all be traced back to a base class of `HyperBinary`, with its common methods shared among all sub-classes.

In [20]:
help(HyperBinary)

Help on class HyperBinary in module kongming.ext.hv:

class HyperBinary(kongming.ext.go.GoClass)
 |  HyperBinary(*args, **kwargs)
 |
 |  HyperBinary models an immutable sparse binary hyper-vector.
 |
 |  Method resolution order:
 |      HyperBinary
 |      kongming.ext.go.GoClass
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, *args, **kwargs)
 |      handle=A Go-side object is always initialized with an explicit handle=arg
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  cardinality(self)
 |      Cardinality() int
 |
 |  clone(self)
 |      Clone() object
 |
 |  core(self)
 |      Core() object
 |
 |  exponent(self)
 |      Exponent() int
 |
 |  model(self)
 |      Model() int
 |
 |  power(self, p)
 |      Power(int p) object
 |
 |  proto_load(self, ctx)
 |      ProtoLoad(object ctx) object, str
 |
 |  repr(self, opts)
 |      Repr(long opts) str
 |
 |  stable_hash(self)
 |      StableHash() long
 |
 |  string(self)
 |      String() str
 |
 |  width(sel

`HyperBinary.core()` justifies a bit special attention here, as it will return the underlying raw data for inspection.

In [21]:
core_a = a.core()
type(core_a)

kongming.ext.hv.SparseSegmented

For `core_a` (which is of type `SparseSegmented`), it has a method of `.offsets()` to return the offsets for all segments.

In [22]:
core_a.offsets()

go.Slice_uint32([200, 850, 808, 971, 988, 507, 623, 163, 386, 401, 436, 272, 209, 758, 132, 148, 606, 12, 54, 935, 944, 279, 651, 295, 406, 697, 57, 615, 769, 732, 133, 941, 784, 679, 599, 927, 428, 348, 776, 386, 242, 344, 306, 607, 8, 904, 303, 1014, 315, 817, 557, 257, 247, 213, 170, 570, 443, 246, 646, 139, 382, 719, 41, 96, 227, 845, 634, 881, 728, 973, 633, 59, 239, 544, 626, 208, 548, 381, 901, 719, 15, 412, 319, 653, 156, 561, 856, 478, 375, 171, 609, 344, 247, 934, 668, 855, 813, 775, 746, 650, 581, 308, 205, 910, 596, 567, 577, 365, 781, 321, 154, 274, 1020, 284, 692, 658, 694, 731, 269, 404, 95, 739, 402, 402, 415, 741, 0, 427, 848, 1007, 175, 937, 451, 999, 878, 24, 249, 208, 608, 526, 366, 252, 561, 788, 1021, 263, 188, 690, 859, 873, 802, 702, 413, 804, 583, 90, 546, 959, 448, 861, 346, 522, 291, 207, 226, 457, 516, 890, 113, 1018, 738, 782, 900, 800, 839, 464, 804, 180, 283, 743, 470, 607, 200, 1023, 900, 892, 439, 749, 103, 473, 201, 908, 1023, 132, 992, 786, 546, 735, 

Conceptually it's a list, and you can turn it into Python list with `list()` constructor.

In [23]:
a.stable_hash(), core_a.stable_hash()

(18068566476371935525, 18068566476371935525)

The stable hash is effectively the hash for the underlying offsets. `a` and `core_a` has the same offsets, and thus the same stable_hash.

Another useful way for inspection (and serialization, if necessary) is to convert the `SparseBinary` instances into protobuf messages (or messages in short).

In [24]:
msg=to_message(a)
msg

model: MODEL_1M_10BIT
stable_hash: 18068566476371935525
pod {
  seed: 12968566895078608129
}

msg is the serialized form of vector a. We can verify their signature hashes are identical.

In [25]:
a.stable_hash() == msg.stable_hash

True

msg can be de-serialized back into a Python object:

In [26]:
back=from_message(msg)
back.string()

'✨:🫛0x..0d01'

In [27]:
equal(back, a)

True

We've verified the conversion from serialized form to its original vector was successful, and there is no information loss.

For hypervector `d` and `e`...

In [28]:
(to_message(d), to_message(e))

(model: MODEL_1M_10BIT
 stable_hash: 6204684621203935542
 pod {
   word: "random"
 },
 model: MODEL_1M_10BIT
 stable_hash: 4706140273670025290
 pod {
   word: "RANDOM"
 })

### The bundle operator

`kongming.hv` also provides operations `bundle()`. The first argument is the seed for bundle operation: different seeds will produce different but all valid results.

In [29]:
bundled0=bundle(Seed128.create(0, 0), a, c)

overlap(bundled0, a), overlap(bundled0, c)

(507, 518)

As expected, the overlap is approximately half of the total cardinality (count of ON bits), for bundling of 2 hyper-vectors: the original vector `a` and `c` (with the model of `MODEL_1M_10BIT` has precisely 1024 ON bits.

`Seed128.create()` returns a 128bit seed for the bundle operation: it takes two uint64 seeds.

With a different seed to bundle the same `a` and `c`, we will have different result, but the overlaps are similar.

In [30]:
bundled1=bundle(Seed128.create(0, 1), a, c)
overlap(bundled1,a), overlap(bundled1,c)

(524, 501)

In [31]:
equal(bundled0, bundled1)

False

If we take a closer look at the resulted `bundled0`...

The Python type for `bundle0` is `Parcel`, which contains the detailed information about the bundle operation. Turn this into a message can make inspection more easier.

In [32]:
to_message(bundled0)

hint: PARCEL
model: MODEL_1M_10BIT
stable_hash: 13546911524086183373
parcel {
  parts {
    model: MODEL_1M_10BIT
    contains {
      pod {
        seed: 12968566895078608129
      }
    }
    contains {
      pod {
        seed: 1234
      }
    }
    checksum: 8230572650216612631
  }
}

In [33]:
to_message(bundled1)

hint: PARCEL
model: MODEL_1M_10BIT
stable_hash: 8112248548191901944
pod {
  seed: 1
}
parcel {
  parts {
    model: MODEL_1M_10BIT
    contains {
      pod {
        seed: 12968566895078608129
      }
    }
    contains {
      pod {
        seed: 1234
      }
    }
    checksum: 8230572650216612631
  }
}

As expected, `bundle0` and `bundle1` are different due to different seed. The seed are recorded in `domain`/`pod` field.

There is no `domain` nor `pod` field in `bundle0` as they take the default value of 0.
There is no `domain` field in `bundle1` as it takes the default value of 0.

### The bind operator

Another critical operation for hyper-dimensional vectors are `bind()`.

In [34]:
bound=bind(a, c)

The bound vector will have almost no overlap with original vectors:

In [35]:
overlap(bound, a), overlap(bound, c)

(0, 1)

Now we take a closer look at the resulted `bound`, which has the type of `Knot`.

In [36]:
type(bound)

kongming.ext.hv.Knot

The type for `bound` is `Knot`, which contains all information about the arguments. Turn this into a message can make inspection more easier.

In [37]:
to_message(bound)

hint: KNOT
model: MODEL_1M_10BIT
stable_hash: 16606468074517378174
knot {
  parts {
    model: MODEL_1M_10BIT
    contains {
      pod {
        seed: 12968566895078608129
      }
    }
    contains {
      pod {
        seed: 1234
      }
    }
    checksum: 8230572650216612631
  }
}

### Domains and pods

We talked about domains earlier, which is basically a group of hypervectors that shares the same logical meaning.

`d0()` refers to the default domain, `Domain.from_name()` will return a named domain and `Domain.from_id()` returns a domain with non-default id.

In [38]:
d1=Domain.from_name("my_domain")
d2=Domain.from_id(2)

str(d1), str(d2)

('🔗my_domain', '🌐0x..0002')

🔗 implies the domain by its name, while 🌐 implies domain by its last 4 digits of hexidecimal id.

As an exercise, you can re-create above hypervectors such as `a`, `b`, `c` in non-default domain of `d1` and `d2`.

On the other hand, pods are just slots within the same domain. Mnemonically a string or a pre-defined enum can also provide such a uint64 pod.

In [39]:
p0=Pod.from_seed(1234)
p1=Pod.from_prewired(api.Prewired.E.STEP)
p2=Pod.from_word("first_pod")

str(p0), str(p1), str(p2)

('🫛0x..04d2', '🍀𓊍', '🌱first_pod')

🫛 implies pod by the last 4 digits of its hexidecimal id, 🍀 implies pod by its pre-defined enum, and 🌱 implies pod by its name.

The pair of `domain` / `pod` can effectively act as the unique identifier for that hypervector.

In addition, most hypervector sub-classes will need an underlying bundle operation, with 128bit seed. We will re-use `domain` / `pod` pair for mnemonic assistance, as each is fundemantally a 64bit numeric value.

## Composite structures

We can produce composites (such as sets, sequences) with hypervectors. The resutled composites are immutable. 




### Sets

Sets are constructed by:

$$ S = S_{marker} \otimes ( \sum_{i, \oplus} {M_i} )$$

where $S_{marker}$ is a set-specific hypervector, determined by a domain: recall domain is a logical group of hypervectors. All sets within the same domain will have the same marker, while different domain will have different markers.

For example, we use a non-default domain of `d1` here.

In [40]:
set0 = Set.create(d1, Pod.from_seed(1234), a, b, c, d)

The resulted `set0` is of type `Set`, as shown below.

In [41]:
to_message(set0)

hint: SET
model: MODEL_1M_10BIT
stable_hash: 6482077344286540886
domain {
  name: "my_domain"
}
pod {
  seed: 1234
}
set {
  members {
    model: MODEL_1M_10BIT
    contains {
      pod {
        seed: 12968566895078608129
      }
    }
    contains {
      pod {
        seed: 17401969588651908185
      }
    }
    contains {
      pod {
        seed: 1234
      }
    }
    contains {
      pod {
        word: "random"
      }
    }
    checksum: 4565871301187706235
  }
}

Let's check to ensure `set0` are really a set composed of `a`, `b`, `c`, and `d`.

In [42]:
set0_marker = Sparkle.from_prewired(so.model(), d1, api.Prewired.SET_MARKER)
set0_stripped = bind(set0, inverse(set0_marker))

(overlap(set0_stripped, a), overlap(set0_stripped, b),
 overlap(set0_stripped, c), overlap(set0_stripped, d))

(269, 238, 279, 238)

In order to get the proper overlap, we need to strip off the extra marker from the set code.

### Sequences

Sequences are constructed by:

$$ S = S_{marker} \otimes ( \sum_{i, \oplus} {M_i} \otimes S_{step}^{i}) $$

where $S_{marker}$, and $S_{step}$ are sequence-specific hypervectors, determined by a domain: recall domain is a logical group of hypervectors. All sets within the same domain will have the same marker, while different domain will have different markers.

For example, we use a non-default domain of d1 here.

In [43]:
seq0 = Sequence.create(d1, Pod.from_seed(1234), a, b, c, d)

The resulted `seq0` is of type `Sequence`, as shown below.

In [44]:
to_message(seq0)

hint: SEQUENCE
model: MODEL_1M_10BIT
stable_hash: 6690773973332605900
domain {
  name: "my_domain"
}
pod {
  seed: 1234
}
sequence {
  members {
    model: MODEL_1M_10BIT
    contains {
      pod {
        seed: 12968566895078608129
      }
    }
    contains {
      pod {
        seed: 17401969588651908185
      }
    }
    contains {
      pod {
        seed: 1234
      }
    }
    contains {
      pod {
        word: "random"
      }
    }
    checksum: 4565871301187706235
  }
}

Let's check to ensure `seq0` is really a sequence of `a`, `b`, `c` and `d`.

In [45]:
seq0_marker = Sparkle.from_prewired(so.model(), d1, api.Prewired.SEQUENCE_MARKER)
seq0_stripped = bind(seq0, inverse(seq0_marker))

step = Sparkle.from_prewired(so.model(), d1, api.Prewired.STEP)

(overlap(seq0_stripped, a), overlap(bind(seq0_stripped, step.power(-1)), b),
 overlap(bind(seq0_stripped, step.power(-2)), c), overlap(bind(seq0_stripped, step.power(-3)), d))

(269, 239, 280, 240)

### Key-value pairs

Key-pair pairs (or maps) are constructed by:

$$ S = S_{marker} \otimes (\sum_{i, \oplus} {K_i \otimes M_i}) $$

where $K_i$ is the `Sparkle` code as seeded by the key string.

Again, $S_{marker}$ is domain-specific hypervectors: all sets within the same domain will have the same marker, while different domain will have different markers.

In [46]:
octopus0 = Octopus.create(d1, Pod.from_seed(1234), keys(["first", "second", "third", "fourth"]), a, b, c, d)

Note we need to use `keys` to pass a native Python slice into this package.

The produced key-value pairs (or maps) will have the type of `Octopus`.

In [47]:
to_message(octopus0)

hint: OCTOPUS
model: MODEL_1M_10BIT
stable_hash: 3508622058760239433
domain {
  name: "my_domain"
}
pod {
  seed: 1234
}
octopus {
  keys: "first"
  keys: "second"
  keys: "third"
  keys: "fourth"
  values {
    model: MODEL_1M_10BIT
    contains {
      pod {
        seed: 12968566895078608129
      }
    }
    contains {
      pod {
        seed: 17401969588651908185
      }
    }
    contains {
      pod {
        seed: 1234
      }
    }
    contains {
      pod {
        word: "random"
      }
    }
    checksum: 4565871301187706235
  }
}

Let's check to ensure `octopus0` is really a key-value pair collections.

In [48]:
octopus0_marker = Sparkle.from_prewired(so.model(), d1, api.Prewired.OCTOPUS_MARKER)
octopus0_stripped = bind(octopus0, inverse(octopus0_marker))

key0 = Sparkle.from_word(so.model(), d1, "first")
key1 = Sparkle.from_word(so.model(), d1, "second")
key3 = Sparkle.from_word(so.model(), d1, "third")
key4 = Sparkle.from_word(so.model(), d1, "fourth")

(overlap(octopus0_stripped, bind(key0, a)), overlap(octopus0_stripped, bind(key1, b)),
 overlap(octopus0_stripped, bind(key3, c)), overlap(octopus0_stripped, bind(key4, d)))

(269, 239, 280, 239)

## Learners

Learners are mutable hypervectors that learn over time, on a stream of observations.

Anothe way to visualize it is that it's a running average over incoming observations.

Two convenience functions can be used to create a new learner...

In [49]:
learner0 = Learner.create(so.model(), d0(), Pod.from_seed(1234))

`Learner.create` takes a model and a seed as its initializer.

In [50]:
learner1 = Learner.random(so)

This will create a learner from the `so` instance, which will provide both model and initial seed.

In [51]:
type(learner0), type(learner1)

(kongming.ext.hv.Learner, kongming.ext.hv.Learner)

The generated learners are (roughly) of type `Learner`.

In [52]:
str(learner0)

'💫{,age=0}'

A newly-created learner instance has the age of $0$, meaning it hasn't received any observation yet. The underlying code is the identity vector (for that model). 

We can add one observation with `bundle()`, like this:

In [53]:
learner0.bundle(a)

''

In [54]:
str(learner0)

'💫{📝0x..4125,age=1}'

💫 implies this is a `Learner` instance, and 📝 is followed by the hash of the underlying hypervector. The age is increased by 1 after a single observation.

Now the `learner0` should contain the whole observation of `a`, and we can verify this by `weight()`, which returns the normalized overlap between a probe code and the learner.

In [55]:
learner0.weight(a), overlap(learner0, a), equal(learner0, a)

(1.0, 1024, True)

What happens if we have a streaming input of `b`, `c` and `d`...

In [56]:
learner0.bundle(b)
learner0.bundle(c)
learner0.bundle(d)

''

In [57]:
learner0.weight(a), overlap(learner0, a), equal(learner0, a)

(0.24144672531769307, 248, False)

This is expected as so far `a` only appears 1 out of 4 obsevations. 

What happens if we observe `a` in the stream again...

In [58]:
learner0.bundle(a)

''

In [59]:
learner0.weight(a), overlap(learner0, a), equal(learner0, a)

(0.3919843597262952, 402, False)

As expected, `a` now accounts for 2 out of 5 observations so far.

`bundle_multiple()` can be used to add impressions multiple times.

In [60]:
learner0.bundle_multiple(b, 3)

''

In [61]:
learner0.weight(b), overlap(learner0, b)

(0.5112414467253177, 524)

This is expected as so far `b` occurs 4 out of the observation stream `a`, `b`, `c`, `d`, `a`, `b`, `b`, `b`.